In [103]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile
import cPickle as pickle
import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import bisect
import matplotlib
matplotlib.use('Agg')
import tensorflow as tf
import matplotlib.pyplot as plt
import argparse
import subprocess as sp

# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'

# start = 640000

def load_test_file(test_set_file):
  rows = []
  with open(test_set_file, 'r') as f:
    for idx, line in enumerate(f):
      row = line.strip().replace(';', ',').split(',')
      rows.append(row)
    return rows

def test(rows, dictionary, embeddings_U):
  score1, score2 = [], []
  for idx, row in enumerate(rows):
    if row[0] in dictionary and row[1] in dictionary:
      score1.append(float(row[2]))
      word1 = dictionary[row[0]]
      word2 = dictionary[row[1]]
      score2.append(embeddings_U[word1,:].dot(embeddings_U[word2,:].T) / (np.linalg.norm(embeddings_U[word1,:], 2) * np.linalg.norm(embeddings_U[word2,:], 2)))
  return score1, score2

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception(
        'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)


# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

def build_dataset(words, n_words, with_UNK = True, shuffle = False, count = None):
  """Process raw inputs into a dataset."""
  if count is None:
    if with_UNK:
      count = [['UNK', -1]]
      count.extend(collections.Counter(words).most_common(n_words - 1))
    else:
      count = []
      count.extend(collections.Counter(words).most_common(n_words))

    if shuffle:
      count = np.random.permutation(count)
    else:
      count = count
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
      data.append(index)
    else:
      index = dictionary['UNK']
      unk_count += 1
      if with_UNK:
        data.append(index)
  if with_UNK:
    count[dictionary['UNK']][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

def build_cooccurance_dict(data, count, dictionary, reverse_dictionary, skip_window):
  cooccurance_count = collections.defaultdict(collections.Counter)
  for idx, center_word in enumerate(data):
    center_word_id = center_word
    if idx >= skip_window - 1 and  idx < len(data) - skip_window:
      for i in range(skip_window):
        cooccurance_count[center_word_id][data[idx-i-1]] += 1
        cooccurance_count[center_word_id][data[idx+i+1]] += 1
    elif idx < skip_window - 1:
      for i in range(skip_window):
        cooccurance_count[center_word_id][data[idx+i+1]] += 1
      for i in range(idx):
        cooccurance_count[center_word_id][data[i]] += 1
    else:
      for i in range(skip_window):
        cooccurance_count[center_word_id][data[idx-i-1]] += 1
      for i in range(idx+1, len(data)):
        cooccurance_count[center_word_id][data[i]] += 1
  return cooccurance_count

def rSquared(coeffs, x, y):
  p = np.poly1d(coeffs)
  yhat = p(x)                      # or [p(z) for z in x]
  ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
  ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
  sstot = np.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
  return ssreg / sstot

def getExponents(spectrum, x):
  m, b = np.polyfit(x, np.log(spectrum), 1)
  print(rSquared([m, b], x, np.log(spectrum)))
  return m, b

def softThreshold(spectrum, sigma):
  return (spectrum - 2 * np.sqrt(len(spectrum)) * sigma).clip(min=0)

def plotFit(coeffs, x, y, name):
  m, b = coeffs
  fig = plt.figure()
  plt.plot(x, y, '.')
  plt.plot(x, np.exp(m*x+b), '-')
  plt.xlabel('dimensionality')
  plt.ylabel('log of spectrum')
  plt.yscale('log')
  fig.tight_layout()
  plt.savefig('{}.pdf'.format(name))

Found and verified text8.zip


In [104]:
vocabulary = read_data("text8.zip")
print('Data size', len(vocabulary))

vocabulary_size = 10000
size = None

start = np.random.randint(0, len(vocabulary)-1)
if size is None:
  train_set = vocabulary
else:
  train_set = vocabulary[start: min(len(vocabulary), start + size)]
  train_set += vocabulary[0: start + size - min(len(vocabulary), start + size)]


data, count, dictionary, reverse_dictionary = build_dataset(train_set,
vocabulary_size)
vocabulary_size = min(vocabulary_size, len(count))

print('Top 10 most freq word for train data: {}'.format(count[:10]))
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

skip_window = 5
cooccur = build_cooccurance_dict(data, count, dictionary, reverse_dictionary, skip_window)
# Do SVD
Nij = np.zeros([vocabulary_size, vocabulary_size])
for i in range(vocabulary_size):
  for j in range(vocabulary_size):
    Nij[i,j] += cooccur[i][j]
Ni = np.zeros(vocabulary_size)
for item in count:
  Ni[dictionary[item[0]]] = item[1]

Data size 17005207
Top 10 most freq word for train data: [['UNK', 1737307], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Most common words (+UNK) [['UNK', 1737307], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


The 4 smaller datasets can be processed in memory  
The English Wikipedia is processed separately, and  
the counts Nij are extracted and saved in a pickle file.

In [105]:
# if not for the english wikipedia corpus,
# comment out the below two lines
# with open('Nij_wikipedia.pkl', 'r') as f:
#   Nij = pickle.load(f)
tot = np.sum(Nij)
print(tot)
print(np.sum(Ni))
Pij = Nij / tot 
Pi = Ni / np.sum(Ni)

170052041.0
17005207.0


Now, check how good the region of interest decays exponentially  
We do this by checking all the matrices: PPMI, PMI and log count.  
To estimate the spectrum, we use the method proposed by Chaterjee: $\lambda_i = (\tilde{\lambda_i}-2\sqrt{n}\sigma)_+$  
where $\tilde{\lambda_i}$ is the ith empirical eigenvalue.
The mysterious 0.35 and 0.138 are the estimated noise standard deviation, c.f.  
https://arxiv.org/abs/1803.00502

In [106]:
log_count = np.log(Nij + 1)
U_lc, D_lc, V_lc = np.linalg.svd(log_count)

In [107]:
PMI = np.zeros([vocabulary_size, vocabulary_size])
for i in range(vocabulary_size):
  for j in range(vocabulary_size):
    if Pi[i] * Pi[j] > 0 and Pij[i,j] > 0:
      PMI[i,j] = np.log(Pij[i,j] / (Pi[i] * Pi[j]))

PPMI = np.maximum(PMI, 0.0)
U, D, V = np.linalg.svd(PPMI)

In [108]:
U_pmi, D_pmi, V_pmi = np.linalg.svd(PMI)

In [109]:
D_ = softThreshold(D_pmi, 0.35)
rank = 0
for idx in range(D_.shape[0]):
  if D_[idx] == 0:
    rank = idx
    break
print("rank={}".format(rank))
D_ = D_[150:1500]
x = np.array(range(150, 1500))
coeffs = getExponents(D_, x)
plotFit(coeffs, x, D_, 'exp_fit_pmi')

rank=2369
0.9894326602916788


In [110]:
D_ = softThreshold(D, 0.35)
rank = 0
for idx in range(D_.shape[0]):
  if D_[idx] == 0:
    rank = idx
    break
print("rank={}".format(rank))
D_ = D_[150:1500]
x = np.array(range(150, 1500))
coeffs = getExponents(D_, x)
plotFit(coeffs, x, D_, 'exp_fit_ppmi')

rank=2253
0.9917396798583351


In [113]:
D_ = softThreshold(D_lc, 0.13)
rank = 0
for idx in range(D_.shape[0]):
  if D_[idx] == 0:
    rank = idx
    break
print("rank={}".format(rank))
D_ = D_[150:1500]
x = np.array(range(150, 1500))
coeffs = getExponents(D_, x)
plotFit(coeffs, x, D_, 'exp_fit_logcount')

rank=2922
0.9916167049819444


In [112]:
import subprocess as sp
sp.check_output("python ../word2vec/send_note.py", shell=True)

''